In [1]:
import pandas as pd
import re
from unidecode import unidecode
import datetime
import plotly.express as px

pd.set_option('max_colwidth', None)

In [2]:
df = pd.read_csv('dataset\csv\coronavirus_comments.csv')

In [3]:
df.columns

Index(['id', 'author', 'body', 'score', 'total_awards_received', 'created_utc',
       'parent_id', 'permalink'],
      dtype='object')

In [4]:
# Do not need parent_id and permalink
df.drop(['parent_id', 'permalink'], axis=1, inplace=True)

In [5]:
df.head(5)

,id,author,body,score,total_awards_received,created_utc
0,ff4r0j3,GreedyLeadership1,yes,1,0,1579570358
1,ff4vilk,fairfax80,what about all the e-commerce happening on AliExpress? Can products coming from China be contaminated?,1,0,1579573443
2,ff4ylul,arm_channel,"That depends on many factors. If someone who is contaminated the goods handled the products, then the answer would be yes. Otherwise the likelihood is low especially if the goods are not perishable products.",1,0,1579575582
3,ff5akx0,DennisShapiro,"Ahhh sweet sweet purge !!\n\n Ever played ""plague inc.""\n\nIt's fun! You get to be a virus and evolve ahead of the WHO and infect and destroy the world! \n\nCoronavirus is respiratory. Hard to go global with it. Could be done.. Hmm I'll boot up the app and design and get back to ya",1,0,1579585315
4,ff5o76t,CLO_Junkie,/r/China_Flu,1,0,1579602851


# Label Data
-----

In [12]:
df[df['body'].isna()].shape

(22, 6)

In [49]:
# Replace NAN in body with empty string
df['body'].fillna('', inplace=True)

In [50]:
# Label deleted comments
df.loc[df['body'] == '[deleted]', 'label'] = 'deleted'

In [51]:
# Label removed comments
df.loc[df['body'] == '[removed]', 'label'] = 'removed'

In [52]:
# Label bot comments
df.loc[df['author'] == 'AutoModerator', 'label'] = 'bot'
df.loc[df['author'].str.contains(r'[bB][0oO][tT]\b'), 'label'] = 'bot'
df.loc[df['author'] == 'dadbot_2', 'label'] = 'bot'
df.loc[df['author'] == 'coronabot22', 'label'] = 'bot'
df.loc[df['author'] == 'B0tRank', 'label'] = 'bot'
df.loc[df['body'].str.contains(r'comment has been removed'), 'label'] = 'bot'
df.loc[df['body'].str.contains(r'submission has been removed'), 'label'] = 'bot'
df.loc[df['body'].str.contains(r'post has been removed'), 'label'] = 'bot'

In [56]:
# Label duplicate data

# Keep single copy of duplicate with max score and label the remaining duplicates.
score_maxes = df.groupby(['author', 'body']).score.transform(max)
df.loc[(df.score != score_maxes) & df.label.isna(), 'label'] = 'duplicate'
# If there is no max, keep one and label the remaining as duplicate.
df.loc[df.duplicated(['author', 'body'], keep='first') & df.label.isna(), 'label'] = 'duplicate'

In [66]:
# Label remaining data
df.loc[df['label'].isna(), 'label'] = 'comment'

# Data Cleaning of 'body'
-----

In [72]:
df2 = df[df.label == 'comment']

In [74]:
# Remove \n
df2['body'].replace(regex=r'(\n)+', value='', inplace=True)

In [75]:
# Remove URLs
df2['body'].replace(regex=r'http\S+|www\S+', value='', inplace=True)

In [78]:
df2['body'].replace(regex={r'&amp;': '&', r'&lt;': '<', r'&gt;': '>'}, inplace=True)
df2['body'].replace(regex={r'\s&\s': ' and '}, inplace=True)

In [79]:
# Function to replace/remove HTML encodings
from io import StringIO
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [84]:
# Remove HTML tags
df2['body'] = df2['body'].map(strip_tags)

In [85]:
# Remove parenthesis (), brackets [], {} and the contents within
df2['body'].replace(regex=r'[([{].*[)\]}]', value='', inplace=True)

In [87]:
# Replace curse words
df2['body'].replace(regex={r'(s|S)[h$@#*%!]{2}(t|T)': 'shit', r'(f|F)[$@#*%!cC]{2}[$@#*%!kK]': 'fuck', r'(b|B)ulls?[^\d\s]{3,}': 'bullshit'}, inplace=True)

In [88]:
# Standardise 'US' and 'UK'
df2['body'].replace(regex={r'\b(USA|U.S.A|u.s|usa)\b': 'US', r'\b(uk|U.K|u.k)\b': 'UK'}, inplace=True)

In [89]:
# Replace email addresses
df2['body'].replace(regex={r'\S+@\w+(\.\w+)+': ''}, inplace=True)

In [93]:
# Remove all single and double quotes, except contractions
df2['body'].replace(regex={r'[\'\"]\B|\B[\'\"]|(\d+[\'\"])+(\d*)': ''}, inplace=True)

In [99]:
# Remove the rest of special characters and digits
df2['body'].replace(regex={r'[^A-Za-z !?\']': ' '}, inplace=True)

In [100]:
# Remove excess whitespace
df2['body'] = df2['body'].str.strip()
df2['body'].replace(regex={r'\s{2,}': ' '}, inplace=True)

In [103]:
# Remove rows with empty strings
df2.loc[df2.body == '', 'label'] = 'empty'

,id,author,body,score,total_awards_received,created_utc,label
82,ffc8u9e,JohanesYamakawa,,1,0,1579794180,comment
124,ffcsran,bunnypirateholly,,1,0,1579806506,comment
134,ffcxdz4,pummers88,,1,0,1579809253,comment
239,ffdkdxg,redditknees,,1,0,1579822735,comment
244,ffdm0ot,kimmey12,,1,0,1579823734,comment
...,...,...,...,...,...,...,...
9411809,glgve5w,Viewfromthe31stfloor,,-1,0,1612088664,comment
9412094,glgzwxu,Viewfromthe31stfloor,,2,0,1612093114,comment
9412200,glh1zdh,Pixel3aXL,,1,0,1612095081,comment
9412259,glh33fd,Charlatanism,,3,0,1612096101,comment


In [116]:
df.update(df2.label)

## Visualisation of comment dataset
-----

In [14]:
df_pie = df.groupby('label').size().reset_index(drop=False, name='size')

In [15]:
df_pie

,label,size
0,bot,322700
1,comment,8499244
2,deleted,120402
3,duplicate,87550
4,empty,60831
5,removed,322604


In [57]:
fig = px.pie(df_pie, values='size', names='label', color='label', color_discrete_sequence=px.colors.sequential.thermal, title='Comment distribution', width=500, height=500)

fig.show()

-----

In [130]:
df_comments = df2[df.label == 'comment']

In [122]:
def timestamp_to_yearmonth(ts):
    return datetime.datetime.fromtimestamp(ts).strftime('%Y-%m')
def timestamp_to_week(ts):
    return datetime.datetime.fromtimestamp(ts).strftime('%Y-Wk%W')

In [ ]:
df_comments['month'] = df_comments['created_utc'].map(timestamp_to_yearmonth)
df_comments['week'] = df_comments['created_utc'].map(timestamp_to_week)

In [136]:
df_comments.to_pickle('comments_processed_preSA.pkl')